<a href="https://colab.research.google.com/github/iodoform/HandFontMaker/blob/main/HandFontMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 手書きフォントメーカー
## 概要
タブレット、スマホ、パソコン等で手書きフォントが作成できるプログラム。
## 使い方
1. 下にあるフォント範囲選択で、フォントを作成したい文字範囲のチェックボックスを選択してください。複数選択可能。
1. メニューから`ランタイム＞すべてのセルを実行` を選択。
1. Google Drive のアクセスを求められるので、許可してください。
1. しばらく待つと、一番下にフォントを書き込みUIが表示されるので、「現在の文字」と書かれている文字を手書き入力画面に書き込み、`次の文字` ボタンを押してください。手書き入力画面には灰色の部分と白色の部分がありますが、灰色部分はフォントに反映されません。

  たまに入力画面の背景の補助線画像がずれて表示されることがありますが、そういった場合は`リロード`ボタンを押してください。何回か押すと直ります。
1. 選択した文字範囲がすべて書き終わると「最後の文字です。フォント作成ボタンを押してください。」と表示されるので、指示に従い`フォント作成`ボタンを押してください。ちなみに、最後まで書かずとも、`フォント作成`ボタンを押すとその時点までのフォントファイルができます。
1. 出来上がったフォントは`/content/drive/MyDrive/fontdir/handfont.otf`からダウンロードできます。

  ### タイポグラフィ
  
    ![Typography Line](https://github.com/iodoform/HandFontMaker/blob/main/Typography_Line_Terms.svg?raw=1)

    フォント用の文字を書くときは、基本的に赤いベースラインよりも上に字を書くということさえ分かっていれば大丈夫です。

    下の「タイポグラフィの補助線設定」という欄からタイポグラフィ用の補助線が設定できますが、タイポグラフィに詳しい方以外は触らないほうがいいです。設定したいときは、上記の使い方手順１と２の間で設定してください。

## 備考
- 一度フォントを作成した後も、`/content/drive/MyDrive/fontdir/svgdir`にはフォントに変換する前のsvgデータが残っているので、再度このプログラムを実行すれば追加のフォントを作成することができます。<br>（使用例：半角英数字だけ作ってあとから全角ひらがなを追加）
- このプログラムの作者([iodoform](https://twitter.com/_iodoform_))は、他の人がこのプログラムを用いて作成したフォントに対して一切の権利を主張しません。

## 謝辞
構成コードは以下の方々のライブラリ、スニペット、コードを参考、あるいは依拠しています。 またnotebookは下記コード群のライセンスを継承します。
- [handfontgen: Handwriting Font Generator](https://github.com/nixeneko/handfontgen.git)
- [font-jp-subset](https://github.com/blivesta/font-jp-subset.git)

## 参考ページ
- https://nixeneko.hatenablog.com/entry/2016/02/06/114348
- https://note.com/npaka/n/nb9d4902f8f4d
- https://www.gradio.app/docs/interface

In [1]:
#画像をフォントに変換するのに必要なパッケージを追加
!apt-get update
!apt-get install fontforge
!apt-get install python3-fontforge
!apt-get install potrace
!apt-get install zbar-tools libcairo2-dev python3-numpy python3-cairosvg python3-pypdf2 python3-pip libffi-dev
!pip3 install qrcode

!git clone https://github.com/nixeneko/handfontgen.git
%cd handfontgen/handfontgen
# 文字範囲のサブセット
!git clone https://github.com/blivesta/font-jp-subset.git
# GUIに必要なパッケージを追加
%pip install gradio


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [858 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [979 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [897 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,238 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,129 kB]
Hit:13 https://ppa.launchpad

In [93]:
#@markdown # フォント文字範囲選択
# 文字範囲を指定
#@markdown 英数字
Alphanumeric = True #@param {type:"boolean"}
#@markdown ひらがな
Hiragana = False #@param {type:"boolean"}
#@markdown カタカナ
Katakana = False #@param {type:"boolean"}
#@markdown 記号
Kigou = False #@param {type:"boolean"}
#@markdown 漢字JIS第一水準
KanjiJIS1 = False #@param {type:"boolean"}
#@markdown 全角英数字
Zenkakueisu = False #@param {type:"boolean"}

fontChar = ""

if Alphanumeric == True:
    with open("/content/handfontgen/handfontgen/font-jp-subset/characters/ASCII.txt") as f:
        fontChar+=f.read()
if Hiragana == True:
    with open("/content/handfontgen/handfontgen/font-jp-subset/characters/ひらがな.txt") as f:
        fontChar+=f.read()
if Katakana == True:
    with open("/content/handfontgen/handfontgen/font-jp-subset/characters/カタカナ.txt") as f:
        fontChar+=f.read()
if Kigou == True:
    with open("/content/handfontgen/handfontgen/font-jp-subset/characters/symbol.txt") as f:
        fontChar+=f.read()
if KanjiJIS1 == True:
    with open("/content/handfontgen/handfontgen/font-jp-subset/characters/漢字JIS第1水準.txt") as f:
        fontChar+=f.read()
if Zenkakueisu == True:
    with open("/content/handfontgen/handfontgen/font-jp-subset/characters/全角英数.txt") as f:
        fontChar+=f.read()
fontChar = "Windowsでコンピューターの世界が広がります。1234567890"

In [108]:
import numpy as np
import cv2
#@markdown # タイポグラフィの補助線設定
#@markdown ![Typography Line](Typography_Line_Terms.svg)

baseline = 0.33
int_baseline = int(1000-baseline*1000)
ascender_height = 0.548 #@param {type:"slider", min:0, max:0.67, step:0.01}
int_ascender_height = int(1000-(ascender_height+baseline)*1000)
cap_height = 0.511 #@param {type:"slider", min:0, max:0.67, step:0.01}
int_cap_height = int(1000-(cap_height+baseline)*1000)
x_height = 0.29 #@param {type:"slider", min:0, max:0.67, step:0.01}
int_x_height = int(1000-(x_height+baseline)*1000)
descender_height = -0.196 #@param {type:"slider", min:-0.33, max:0, step:0.01}
int_descender_height = int(1000-(descender_height+baseline)*1000)

baseHankakuImg = np.zeros((1000, 1000, 3))
baseHankakuImg+=210
baseHankakuImg[int_ascender_height:int_descender_height,250:750,:]=255
baseHankakuImg[int_ascender_height-5:int_ascender_height+5,:,:] = 200
baseHankakuImg[int_cap_height-5:int_cap_height+5,:,:] = 200
baseHankakuImg[int_x_height-5:int_x_height+5,:,:] = 200
baseHankakuImg[int_baseline-5:int_baseline+5,:,:] = (255,0,0)
baseHankakuImg[int_descender_height-5:int_descender_height+5,:,:] = 200
baseHankakuImg/=255
baseZenkakuImg = np.zeros((1000, 1000, 3))
baseZenkakuImg+=210
tmpmargin = int((1000-int_baseline+int_ascender_height)/2)
baseZenkakuImg[int_ascender_height:int_baseline,tmpmargin:1000-tmpmargin,:]=255
baseZenkakuImg[int_ascender_height-5:int_ascender_height+5,:,:] = 200
baseZenkakuImg[int_baseline-5:int_baseline+5,:,:] = 200
baseZenkakuImg/=255

In [98]:
# Google Drive をマウント
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/MyDrive/fontdir
!mkdir /content/drive/MyDrive/fontdir/svgdir
outdir = "/content/drive/MyDrive/fontdir/svgdir"
destfile = "/content/drive/MyDrive/fontdir/handfont.otf"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [109]:
import gradio as gr
import numpy as np
import cv2
import subprocess
import scanchars
import sys, os
import glob
import argparse
import slantcorrection
import passzbar
from tilecharbox import Rect
from util import getgrayimage
import passpotrace
import fontgenfromsvg
import unicodedata


metadata = fontgenfromsvg.FontMetaData(
            fontname="TekitounaTegakiFont",
            family="TekitounaTegakiFont",
            fullname="TekitounaTegakiFont",
            weight="Regular",
            copyrightnotice="",
            fontversion="0.01",
            familyJP="適当な手書きフォント",
            fullnameJP="適当な手書きフォント",
            ascent=int(ascender_height*1000),
            descent=int(abs(descender_height)*1000)
            )
currentChar = fontChar[0]
charNum = 0
# 画像をsvgに変換する関数
def makeSVG(input_img):
    global outdir
    global destfile
    global metadata
    global currentChar
    global charNum
    name = "{:04X}".format(ord(currentChar))
    intas=int(ascender_height*1000)
    intds=int(abs(descender_height)*1000)
    margin = int((intas+intds)/2)
    resol = [[margin,1000,margin],[intas,1000,intds]] if isZenkaku(currentChar) else [[250,500,250],[intas,1000,intds]]
    charNum+=1
    try:
        currentChar = fontChar[charNum]
    except IndexError:
        gr.Error("最後の文字です。フォント作成ボタンを押してください。")
    grayimg = getgrayimage(255-input_img["mask"])
    ret, binimg = cv2.threshold(grayimg,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    imgheight = 1000
    margintop   = int(imgheight * resol[1][0] / resol[1][1]) #-h[px] * (margint[mm]/h[mm])
    marginbottom= int(imgheight * resol[1][2] / resol[1][1]) #-h[px] * (marginb[mm]/h[mm])
    imgwidth = int(imgheight * resol[0][1] / resol[1][1]) #h[px] * (w[mm]/h[mm])
    marginleft  = int(imgheight * resol[0][0] / resol[1][1]) #-h[px] * (marginl[mm]/h[mm])
    marginright = int(imgheight * resol[0][2] / resol[1][1]) #-h[px] * (marginr[mm]/h[mm])

    # potrace cannot set size in px when converting to SVG.
    # set size in pt and convert to SVG, after that, replace pt with px
    optargs = [ "-W%dpt"%(imgwidth + marginleft + marginright),
                "-H%dpt"%(imgheight + margintop + marginbottom),
                "-L%dpt"%(- marginleft), "-R%dpt"%(- marginright),
                "-T%dpt"%(- margintop), "-B%dpt"%(- marginbottom)]
    bsvg = passpotrace.passpotrace(binimg,optargs)
    bsvg = bsvg.replace(b"pt", b"px") # any exceptions?

    #save function
    if outdir != '' and not os.path.isdir(outdir):
        os.makedirs(outdir)
    scanchars.saveasfile(outdir, name, bsvg)
    return currentChar
# svgからフォントを生成
def makeFont():
    fontgenfromsvg.generatefont(destfile, metadata, outdir)
# 背景画像の差し替え
def changeBackground():
    global currentChar
    global baseZenkakuImg
    global baseHankakuImg
    return baseZenkakuImg if isZenkaku(currentChar) else baseHankakuImg
# 半角全角判定
def isZenkaku(chr):
    charType = unicodedata.east_asian_width(chr)
    return charType in 'FWA'
# Blocksの作成
with gr.Blocks() as demo:
    output = gr.Text(label = "現在の文字",value = currentChar)
    input = gr.ImageMask(value=baseZenkakuImg,label = "文字入力画面",brush_radius=30,shape=(1000,1000)) if isZenkaku(currentChar) else gr.ImageMask(value=baseHankakuImg,label = "文字入力画面",brush_radius=30,shape=(1000,1000))
    next = gr.ClearButton(input, value = "次の文字")
    next.click(fn = makeSVG, inputs = input, outputs = output).then(fn = changeBackground, outputs=input)
    reLoadBtn = gr.ClearButton(input,value="リロード")
    reLoadBtn.click(fn = changeBackground, outputs=input)
    fontBtn = gr.Button("フォント作成")
    fontBtn.click(fn = makeFont)
# 起動
demo.launch(debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
